In [40]:
import os 
import requests
from scraper import relevant_links,website_content
from groq import Groq

from dotenv import load_dotenv
from IPython.display import Markdown, display


In [41]:
load_dotenv()
api_key=os.getenv("GROQ_API_KEY")

if not api_key:
    print(f"no api key found")

MODEL = 'openai/gpt-oss-120b'

openai=Groq()


In [42]:
links=relevant_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

In [43]:
system_prompt=""" 
    You are provided with a list of links found on a webpage.
    so your work is to find all the relevant links that would help in making the brochure about the company,
    include: About, Mission, Careers, Leadership, Products
    exclude: Terms, Privacy, Contact forms, Email links, Blog articles

    Return **only valid JSON** in this exact schema:
    
    {
        'links':[
            {'type':'about page','url':"https://full.url/goes/here/about"},
            {'type':'careers page','url':"https://another.full.url/careers"}
        ]
    }

"""

In [44]:
def make_userprompt(url):
    user_prompt=f""" 
    here is the list of all the links from this {url} webpage
    You have to decide which links are the relvant links for the brochure of the company.
    Go through all the links thouroughly and help me find the links that would create a good brochure .
    Please respond in json format only.
    
    Do not include Terms of Service, Privacy, email links.
    Links (some might be relative links):
    """
    links=relevant_links(url)
    user_prompt+="\n".join(links)

    return user_prompt

In [45]:
prompt=make_userprompt("https://edwarddonner.com")
print(prompt)

 
    here is the list of all the links from this https://edwarddonner.com webpage
    You have to decide which links are the relvant links for the brochure of the company.
    Go through all the links thouroughly and help me find the links that would create a good brochure .
    Please respond in json format only.

    Do not include Terms of Service, Privacy, email links.
    Links (some might be relative links):
    https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-a

In [46]:
import json
def find_all_relevant_links(url):
    print(f" here we are running this code using this {MODEL} model")
    response=openai.chat.completions.create(
        model=MODEL,
        messages=[
            {'role':'system','content':system_prompt},
            {'role':'user','content':make_userprompt(url)}
        ],
        response_format={'type':'json_object'}
    )
    result=response.choices[0].message.content
    result=json.loads(result)
    print(f" we found {len(result['links'])} relevant links ")
    return result



In [47]:
find_all_relevant_links("https://edwarddonner.com")


 here we are running this code using this openai/gpt-oss-120b model
 we found 4 relevant links 


{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'product page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'product page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'product page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'}]}

In [48]:
print(website_content('https://edwarddonner.com'))

Home - Edward Donner

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Conne

In [49]:
# so now we are gonnna make another call to llm such that it displays the brochure 
# we are gonna do the second step of 

In [50]:
def fetch_links_content(url):

    contents= website_content(url)
    relevant=find_all_relevant_links(url)
    result=f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"

    for i in relevant['links']:
        result+= f"\n ## this is {i['type']} \n"
        result+= website_content(i['url'])

    return result
    

In [51]:
print(fetch_links_content("https://huggingface.co"))


 here we are running this code using this openai/gpt-oss-120b model
 we found 8 relevant links 
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
MiniMaxAI/MiniMax-M2
Updated
4 days ago
•
630k
•
928
deepseek-ai/DeepSeek-OCR
Updated
8 days ago
•
1.85M
•
2.36k
moonshotai/Kimi-Linear-48B-A3B-Instruct
Updated
1 day ago
•
9.06k
•
290
briaai/FIBO
Updated
3 days ago
•
2.46k
•
164
meituan-longcat/LongCat-Video
Updated
4 days ago
•
1.04k
•
254
Browse 1M+ models
Spaces
Running
on
CPU Upgrade
901
901
The Smol Training Playbook: The Secrets to Building World-Class LLMs
📝
Running
15.6k
15.6k
DeepSite v3
🐳
Generate any application by Vibe Coding
Running
2.22k
2.22k
Wan2.2 Animate
👁
Wan2.2 Ani

In [52]:
brochure_system_prompt = """ 
 You are a helpful assistant that is provided with content of the relevant page from company's website_content
 and create a brochure about the company so as to present it to the investors, customers or recruits.
 Respond in markdown without code blocks.
 Include details of company culture,customers, career/jobs . if you have the information.
"""

In [53]:
def brochure_user_prompt(company_name,url):
    user_prompt=f"""
    You are looking at a company called: {company_name}
    Here are the contents of its landing page and other relevant pages;
  
    use this information to build a short brochure of the company in markdown without code blocks.\n\n
    """ 
    user_prompt+=fetch_links_content(url)
    user_prompt=user_prompt[:5000]

    return user_prompt




In [ ]:
def brochure_generator(company_name,url):

    response=openai.chat.completions.create(
        model=MODEL,
        messages= [
            {'role':'system','content':brochure_system_prompt},
            {'role':'user','content':brochure_user_prompt(company_name,url)}
        ],
    )

    result=response.choices[0].message.content
    return result
    

In [55]:
brochure_generator("HuggingFace", "https://huggingface.co")

 here we are running this code using this openai/gpt-oss-120b model
 we found 9 relevant links 


"**Hugging Face Brochure**\n\n**Empowering the Next Generation of Machine Learning**\n\n[Cover Image: Collaborative AI Landscape]\n\nWelcome to Hugging Face, the AI community building the future. At the heart of the AI revolution, we empower engineers, scientists, and end-users to learn, collaborate, and share their work to build an open and ethical AI future together.\n\n**Our Mission**\n\nCreate, discover, and collaborate on ML better. Our platform is the central place where anyone can share, explore, discover, and experiment with open-source ML.\n\n**Key Features**\n\n* **Collaboration Platform**: Host and collaborate on unlimited public models, datasets, and applications.\n* **Fast Innovation**: With the HF Open source stack, move faster and build innovative AI solutions.\n* **Explore All Modalities**: Text, image, video, audio, or even 3D – we support diverse AI applications.\n* **Community**: Join a fast-growing community of 50,000+ organizations and individuals.\n\n**What We Off

In [56]:
def language_brochure_generator(company_name,url,language):
    langauge_generator_prompt=f""" 
    So you are a helpful {language} translator  .
    I am providing you with the brochure of the company .
    Your task is to convert that that brochure into {language} language.
    Keep all markdown headings
    Keep bullet lists
    Keep brand names unchanged

    Return **translated brochure only** — no extra text.

    """
    response=openai.chat.completions.create(
        model=MODEL,
        messages=[
            {'role':'system','content':langauge_generator_prompt},
            {
                'role':'user','content':brochure_generator(company_name,url)
            }
        ]
    )
    result=response.choices[0].message.content
    display(Markdown(result))

In [57]:
language_brochure_generator("HuggingFace", "https://huggingface.co",'spanish')

 here we are running this code using this openai/gpt-oss-120b model
 we found 9 relevant links 


**Hugging Face: Potenciando la Revolución de la IA**

**Sobre Nosotros**

Hugging Face es una plataforma de colaboración para la comunidad de aprendizaje automático, que empodera a la próxima generación de ingenieros, científicos y usuarios finales de ML para aprender, colaborar y compartir su trabajo, construyendo juntos un futuro de IA abierto y ético. Estamos en el corazón de la revolución de la IA, con una comunidad de rápido crecimiento y algunas de las bibliotecas y herramientas de ML de código abierto más utilizadas.

**Nuestra Misión**

Nuestra misión es hacer que la IA sea más accesible y equitativa para todos. Creemos que la IA debe ser una fuerza para el bien y que debe usarse para mejorar la vida de las personas. Nos esforzamos por crear una comunidad abierta y colaborativa donde cada quien pueda contribuir, aprender y crecer juntos.

**Qué Hacemos**

Proporcionamos una plataforma para que la comunidad de aprendizaje automático colabore en modelos, conjuntos de datos y aplicaciones. Nuestra plataforma permite a los usuarios:

* Alojar y colaborar en modelos, conjuntos de datos y aplicaciones públicas ilimitadas  
* Avanzar más rápido con nuestra pila de código abierto  
* Explorar todas las modalidades, incluidos texto, imagen, video, audio y 3D  
* Construir su portafolio y compartir su trabajo con el mundo  
* Acceder a más de 45 000 modelos de los principales proveedores de IA mediante una API única y unificada, sin tarifas de servicio  

**Nuestros Valores**

* **Comunidad**: Creemos en el poder de la comunidad y la colaboración.  
* **Innovación**: Nos esforzamos por innovar y superar los límites de lo que es posible con la IA.  
* **Accesibilidad**: Creemos que la IA debe ser accesible para todos.  
* **Ética**: Nos dedicamos a crear IA responsable y respetuosa de los valores humanos.  

**Nuestros Clientes**

Más de 50 000 organizaciones utilizan nuestra plataforma, entre ellas:

* AI2  
* Team (non-profit)  
* AI at Meta (Enterprise company)  
* Amazon (company)  
* Google (Enterprise company)  
* Intel (company)  
* Microsoft (Enterprise company)  
* Grammarly (Team company)  

**Oportunidades de Carrera**

Somos una empresa de rápido crecimiento con un talentoso equipo científico que explora el límite de la tecnología. Si te apasiona la IA y deseas unirte a una comunidad que está moldeando el futuro de la IA, te invitamos a explorar nuestras ofertas de empleo y postularte para formar parte de nuestro equipo.

**Comienza**

Regístrate de forma gratuita y empieza a explorar nuestra plataforma hoy mismo. Con nuestra interfaz intuitiva y documentación extensa, podrás comenzar en muy poco tiempo.

**Hugging Face Universe**

Encuentra aquí otros recursos disponibles para su uso dentro del universo de la marca Hugging Face.

[Insert Call-to-Action button: ¡Regístrate gratis hoy!]